In [1]:
import pickle

import geopandas as gpd
import osmnx as ox
from scipy.spatial import cKDTree
import networkx as nx

from my_paths import *
import Graph_functions

In [2]:
PATH_STRADE_CICLABILI_PICKLE_RAW = "../Data/Raw/Info_bici/strade_ciclabili.pickle"
PATH_STRADE_CICLABILI_PICKLE_STAGING = "../Data/Staging/Info_bici/strade_ciclabili.pickle"
PATH_STRADE_CICLABILI_GEOJSON_STAGING = "../Data/Staging/Info_bici/strade_ciclabili.geojson"

PATH_CICLABILI_PICKLE_STAGING = "../Data/Staging/Info_bici/ciclabile.pickle"
PATH_CICLABILI_GEOJSON_STAGING = "../Data/Staging/Info_bici/ciclabile.geojson"

PATH_TEST_STAGING = "../Data/Staging/Analisi/"

CRS_GRAD = "EPSG:4326"  # Lat-Long
CRS_METR = "EPSG:32632" # Metri

## Creazione Strade ciclabili Graph (compreso ingestion da openstreet maps)

Ingestion da openstreet maps e salviamo in Raw (pickle)

In [3]:
place = "Milano, Italy"

bike_filter = (
    '["highway"]["bicycle"!~"no"]["bicycle"!~"dismount"]'
)
G_ox_strade = ox.graph_from_place(place, network_type='bike', custom_filter=bike_filter)
with open(PATH_STRADE_CICLABILI_PICKLE_RAW, "wb") as f:
    pickle.dump(G_ox_strade, f)

Carichiamo il file da Row e modifichiamolo

In [3]:
with open(PATH_STRADE_CICLABILI_PICKLE_RAW, "rb") as f:
    G_strade = pickle.load(f)

Aggiungiamo archi di congiunzione artificiali per connettere strade molto vicine tra di loro

In [4]:
G_strade = G_strade.to_undirected()
G_strade = Graph_functions.add_edge_near_nodes(G_strade)

Salva GeoJson & Pickle in staging

In [ ]:
_, gdf_edges = ox.graph_to_gdfs(G_strade)

# Andrebbe pulito il geojson ma per ora lo usiamo solo per visualizzazione su kepler
gdf_edges.to_file(PATH_STRADE_CICLABILI_GEOJSON_STAGING, driver="GeoJSON")

with open(PATH_STRADE_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_strade, f)

## Creazione Ciclabili Graph

Modifico ad hoc il raw delle ciclabili. Se decideremo di utilizzare questo metodo dei graph, modificheremo l'ETL delle ciclabili in questo modo e aggiungeremo la conversione in graph e il salvataggio in pickle

In [3]:
# Carico Raw
gdf_ciclabili_raw = gpd.read_file(PATH_CICLABILI_RAW)
gdf_ciclabili_staging = gdf_ciclabili_raw.copy()

# Convertiamo Multilinestring in Linestring
gdf_ciclabili_staging["geometry"] = gdf_ciclabili_staging["geometry"].apply(lambda x: x.geoms[0])

# Altre operazioni di pulizia. Da considerare di rimuover il group_by dall'ETL delle piste
gdf_ciclabili_staging["fine_lavor"] = gdf_ciclabili_staging["fine_lavor"].astype(int)
gdf_ciclabili_staging["lunghezza"] = gdf_ciclabili_staging["lunghezza"].astype(int)
gdf_ciclabili_staging = gdf_ciclabili_staging.drop(["id_amat", "id_via","gerarchia"], axis=1)

G_ciclabili = Graph_functions.gdf_to_graph(gdf_ciclabili_staging)
with open(PATH_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_ciclabili, f)

Anche qui aggiungiamo connessioni artificiali tra nodi vicini non connessi

In [5]:
# Carichiamo MultiDiGraph da file pickle
with open(PATH_CICLABILI_PICKLE_STAGING, "rb") as f:
    G_ciclabili = pickle.load(f)

# Aggiungiamo archi artificiali
G_ciclabili = G_ciclabili.to_undirected()
G_ciclabili = Graph_functions.add_edge_near_nodes(G_ciclabili)

# Salviamo pickle e geojson in staging
_, gdf_edges = ox.graph_to_gdfs(G_ciclabili)
gdf_edges.to_file(PATH_CICLABILI_GEOJSON_STAGING, driver="GeoJSON") # Anche questo andrebbe ripulito
with open(PATH_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_ciclabili, f)

## Proviamo a creare connessioni con networkx

Carichiamo da file i Graph (dai file pickle)

In [ ]:
# Carichiamo Graph Ciclabili
with open(PATH_CICLABILI_PICKLE_STAGING, "rb") as f:
    G_ciclabili = pickle.load(f)

# Carichiamo Graph Strade
with open(PATH_STRADE_CICLABILI_PICKLE_STAGING, "rb") as f:
    G_strade = pickle.load(f)

# Carichiamo Parchi GeoDataFrame
gdf_parchi = gpd.read_file(PATH_PARCHI_RAW)

# Troviamo i centroid e mettiamoli in un dizionari di Punti come tuple
gdf_parchi["centroid"] = gdf_parchi.geometry.centroid
poi_nodes = {}
for idx, row in gdf_parchi.iterrows():
    poi_nodes[idx] = (row["centroid"].x, row["centroid"].y)

Uniamo in un unico graph